Notebook to locally examine the fiducial KiDS1000 .tfrecords

Based off https://cosmo-gitlab.phys.ethz.ch/jafluri/cosmogrid_kids1000/-/blob/master/kids1000_analysis/input_pipeline.py

In [2]:
import tensorflow as tf
from kids1000_analysis import data, constants, input_pipeline

Instructions for updating:
`MultivariateNormalFullCovariance` is deprecated, use `MultivariateNormalTriL(loc=loc, scale_tril=tf.linalg.cholesky(covariance_matrix))` instead.
Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-02-06 13:20:38.587318: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-06 13:20:38.587451: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
tfr_file = "/Users/arne/data/KiDS1000/fiducial/fiducial_data_000.tfrecord"

In [3]:
dset_signal = tf.data.TFRecordDataset(tfr_file)
dset_signal = data.decode_dset(dset_signal, tag="patch", num_tags=15, shapes=(constants.n_pix_padded, 2*constants.n_z_bins))

# add noise, remove mean, ...

for temp in dset_signal.take(1):
    print(type(temp))
    print(temp.keys())
    print(temp["patch_0"].shape)

2023-02-03 16:13:21.084671: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


<class 'dict'>
dict_keys(['patch_0', 'patch_1', 'patch_10', 'patch_11', 'patch_12', 'patch_13', 'patch_14', 'patch_2', 'patch_3', 'patch_4', 'patch_5', 'patch_6', 'patch_7', 'patch_8', 'patch_9'])
(149504, 10)


# non-batched

In [4]:
dset_concat = data.concat_tag(dset_signal, tag="patch", num_tags=15)

for temp in dset_concat.take(1):
    print(type(temp))
    print(temp.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(2242560, 10)


# batched

In [5]:
dset_batched = dset_signal.batch(3)

for temp in dset_batched.take(1):
    print(type(temp))
    print(temp.keys())
    print(temp["patch_0"].shape)

<class 'dict'>
dict_keys(['patch_0', 'patch_1', 'patch_10', 'patch_11', 'patch_12', 'patch_13', 'patch_14', 'patch_2', 'patch_3', 'patch_4', 'patch_5', 'patch_6', 'patch_7', 'patch_8', 'patch_9'])
(3, 149504, 10)


In [6]:
dset_concat = data.concat_tag(dset_batched, tag="patch", num_tags=15)

for temp in dset_concat.take(1):
    print(type(temp))
    print(temp.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(45, 149504, 10)


# original

In [6]:
local_batch_size = 3
tags = [0]
for i in range(3):
    tags.append(2 * i + 1)
    tags.append(2 * i + 2)


dset = input_pipeline.get_train_dset(
    batch_size=3,
    signal_file_patern="/Users/arne/data/KiDS1000/fiducial/fiducial_data_000.tfrecord",
    noise_file_patern="/Users/arne/data/KiDS1000/fiducial/noise_data_000.tfrecord",
    data_file="/Users/arne/data/KiDS1000/KiDS1000_pixel_data.hdf5",
    with_bary=True,
    m_bias=1,
    c_bias=0,
    node_id=0,
    n_node=1,
    num_readers=1,
    noise_scale=1,
    tags=tags,
    set_for_eval=True,
)


In [9]:
for temp in dset.take(100):
    print(temp.shape)

(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 10)
(21, 149504, 1

In [ ]:
def get_train_dset(batch_size, signal_file_patern, noise_file_patern, data_file, with_bary, remove_mean=True,
                   m_bias=True, c_bias=True, noise_scale=None, tags=None, node_id=0, n_node=1, num_readers=12,
                   file_name_shuffle=128, file_name_shuffle_seed=11, signal_shuffle_buffer=128,
                   signal_shuffle_seed=1111, noise_shuffle_buffer=128, noise_shuffle_seed=111111, set_for_eval=False,
                   prefetch=3):
    """
    Builds the training dataset from given file name patters
    :param batch_size: local batch size, will be multiplied with the number of deltas for the delta loss
    :param signal_file_patern: File pattern to expand to get the signal files
    :param noise_file_patern: File pattern to expand to get the noise files
    :param data_file: KiDS data file for mask building and so on
    :param with_bary: include baryons (changes the number of deltas -> global batch size)
    :param remove_mean: remove the mean from all maps prior to adding the biases
    :param m_bias: add multiplicative bias
    :param c_bias: add additive shear bias
    :param noise_scale: factor to multiply on the noise before addition, should be a tf.Variable
    :param tags: Tags to take, i.e. what parameter perturbations to return, defaults to all
    :param node_id: The id of the current node (for multi GPU training) defaults to a single GPU
    :param n_node: Number of node in the multi GPU training, defaults to 1 (single GPU)
    :param num_readers: Parallel read-out from the files expanded from the patterns, this is parallel over files not
                        samples in the files
    :param file_name_shuffle: shuffle buffer for the files names that were expanded
    :param file_name_shuffle_seed: seed for the file name shuffle (node_id will be added)
    :param signal_shuffle_buffer: Shuffle buffer for the signal samples
    :param signal_shuffle_seed: seed for the signal shuffle (node_id will be added)
    :param noise_shuffle_buffer: Shuffle buffer for the noise samples
    :param noise_shuffle_seed: Seed for the noise shuffle (node_id will be added)
    :param set_for_eval: If this is True, then the dataset won't be shuffled repeated such that one can go through it
                         deterministically exactly once. If this is an int > 0 it will be used as tf.random.set_seed
                         (defaults to 1) to make the bias addition deterministic
    :param prefetch: Prefetch some batches into memory, None -> no prefetching
    :return: A dataset that returns samples with a given batchsize in the right ordering for the delta loss
    """
    # TODO: Think about caching

    # set the seed if necessary
    if set_for_eval:
        if n_node > 1:
            print(f"WARNING: You are sharding the eval version of this dataset across more than one node ({n_node})! "
                  f"The eval version of this different of all number of nodes because of the random state of each node. "
                  f"Be carefeul not to mix different versions!")
        tf.random.set_seed(int(set_for_eval) + node_id)

    # we start with the signal, get the file names shard, shuffle and dataset them
    dset_signal = tf.data.Dataset.list_files(signal_file_patern, shuffle=False)
    dset_signal = dset_signal.shard(n_node, node_id)
    if not set_for_eval:
        dset_signal = dset_signal.repeat()
        dset_signal = dset_signal.shuffle(file_name_shuffle, seed=file_name_shuffle_seed + node_id)

    # interleave
    if set_for_eval:
        dset_signal = dset_signal.interleave(tf.data.TFRecordDataset, cycle_length=num_readers, block_length=1)
    else:
        dset_signal = dset_signal.interleave(tf.data.TFRecordDataset, cycle_length=num_readers, block_length=1,
                                             num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)

    # if we have baryons then the number of tags is higher
    if with_bary:
        num_tags = 19
    else:
        num_tags = 15

    # decode the dset
    if tags is None:
        tags = num_tags
    dset_signal = data.decode_dset(dset_signal, tag="patch", num_tags=tags,
                                   shapes=(constants.n_pix_padded, 2 * constants.n_z_bins))

    # get the masks and bring it into the right shape
    masks = gen_data_masks(data_file=data_file)
    masks = np.concatenate([masks, masks], axis=-1)[None, ...]
    tf_masks = tf.constant(masks, dtype=tf.float32)

    # number of relevant pixels
    n_relevant = np.sum(masks, axis=(0, 1))

    # remove mean (this should actually always happen) with correct weights
    if remove_mean:
        dset_signal = dset_signal.map(lambda x: dset_map_remove_mean(x, weights=constants.n_pix_padded / n_relevant))

    # add biases
    if m_bias or c_bias:
        dset_signal = dset_signal.map(lambda x: dset_map_add_biases(x, m_bias=m_bias, c_bias=c_bias))

    # shuffle and batch
    if not set_for_eval:
        dset_signal = dset_signal.shuffle(signal_shuffle_buffer, seed=signal_shuffle_seed + node_id)
    dset_signal = dset_signal.batch(batch_size)

    # now we deal with the noise, get the file names shard, shuffle and dataset them
    dset_noise = tf.data.Dataset.list_files(noise_file_patern, shuffle=False)
    dset_noise = dset_noise.shard(n_node, node_id)
    if not set_for_eval:
        dset_noise = dset_noise.repeat()
        dset_noise = dset_noise.shuffle(file_name_shuffle)

    # interleave in parallel
    if set_for_eval:
        dset_noise = dset_noise.interleave(tf.data.TFRecordDataset, cycle_length=num_readers, block_length=1)
    else:
        dset_noise = dset_noise.interleave(tf.data.TFRecordDataset, cycle_length=num_readers, block_length=1,
                                           num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)

    # decode the dataset
    dset_noise = data.decode_labeled_dset(dset=dset_noise,
                                          shapes=[(constants.n_pix_padded, 2 * constants.n_z_bins), (2,)],
                                          samples_only=True, return_dict=True)

    # remove the mean
    if remove_mean:
        dset_noise = dset_noise.map(lambda x: dset_map_remove_mean(x, weights=constants.n_pix_padded / n_relevant))

    # shuffle and batch
    if not set_for_eval:
        dset_noise = dset_noise.shuffle(noise_shuffle_buffer, seed=noise_shuffle_seed + node_id)
    dset_noise = dset_noise.batch(batch_size)

    # marry the datasets
    dset_train = tf.data.Dataset.zip((dset_signal, dset_noise))

    # function to add noise + scale the noise according to the noise scale
    if noise_scale is None:
        noise_scale = 1.0

    def add_noise(b1, b2):
        for key in b1.keys():
            b1[key] += noise_scale*b2["sample"]
        return b1

    # map the noise addtion
    dset_train = dset_train.map(add_noise)

    # concat the everything with selected tags
    dset_train = data.concat_tag(dset_train, tag="patch", num_tags=tags)

    # mask again
    dset_train = dset_train.map(lambda x: tf.multiply(x, tf_masks))

    # prefetch
    if prefetch is not None:
        dset_train = dset_train.prefetch(prefetch)

    return dset_train


